In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Common imports
import numpy as np
import pandas as pd
import os, sys, glob  
from os.path import isfile, isdir
import librosa
import librosa.display
import librosa.core as core
import scipy.io.wavfile as wavfile

# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline


import warnings
warnings.filterwarnings("ignore")   # To rid of warnings 

os_sep = os.sep 

if sys.platform == 'win32':   # if windows 
    home = os.path.join('C:', os.sep, 'Users')      
elif sys.platform == "linux" or sys.platform == "linux2" :    
    home = os.path.expanduser("~")   # home = os.getenv("HOME")


In [0]:
wav_data = [
    '/content/drive/My Drive/after_data/20181009_1_100023_person.wav',
    '/content/drive/My Drive/after_data/20181009_3_111450_drone.wav',    
    '/content/drive/My Drive/after_data/20181009_1_101055_person.wav',
    '/content/drive/My Drive/after_data/20181009_3_111734_drone.wav',
    '/content/drive/My Drive/after_data/20181009_1_101420_person.wav',  
    '/content/drive/My Drive/after_data/20181016_1_134334_drone.wav',
    '/content/drive/My Drive/after_data/20181009_1_101635_person.wav', 
    '/content/drive/My Drive/after_data/20181016_1_135124_person.wav',
    '/content/drive/My Drive/after_data/20181009_1_103752_car.wav',     
    '/content/drive/My Drive/after_data/20181016_2_134532_car.wav',
    '/content/drive/My Drive/after_data/20181009_1_104216_car.wav',     
    '/content/drive/My Drive/after_data/20181016_3_142025_person.wav',
    '/content/drive/My Drive/after_data/20181009_1_104446_car.wav',    
    '/content/drive/My Drive/after_data/20181016_3_142531_drone.wav',
    '/content/drive/My Drive/after_data/20181009_2_102248_person.wav',  
    '/content/drive/My Drive/after_data/20181114_1_175502_drone.wav',
    '/content/drive/My Drive/after_data/20181009_2_102508_person.wav',  
    '/content/drive/My Drive/after_data/20181114_2_140400_car.wav',
    '/content/drive/My Drive/after_data/20181009_2_102849_person.wav',
    '/content/drive/My Drive/after_data/20181114_2_141258_car.wav',
    '/content/drive/My Drive/after_data/20181009_3_110458_person.wav',
    '/content/drive/My Drive/after_data/20181114_2_180324_drone.wav',
    '/content/drive/My Drive/after_data/20181009_3_110901_drone.wav',   
    '/content/drive/My Drive/after_data/20181114_3_142450_person.wav'
]

In [0]:
#wav_repo = os.path.join(home, '채진영', 'Desktop', 'counterUAV', 'after_data')
#wav_data = glob.glob(os.path.join(wav_repo,'**','*.wav'), recursive=True)

In [6]:
print(len(wav_data))
wav_data

24


['/content/drive/My Drive/after_data/20181009_1_100023_person.wav',
 '/content/drive/My Drive/after_data/20181009_3_111450_drone.wav',
 '/content/drive/My Drive/after_data/20181009_1_101055_person.wav',
 '/content/drive/My Drive/after_data/20181009_3_111734_drone.wav',
 '/content/drive/My Drive/after_data/20181009_1_101420_person.wav',
 '/content/drive/My Drive/after_data/20181016_1_134334_drone.wav',
 '/content/drive/My Drive/after_data/20181009_1_101635_person.wav',
 '/content/drive/My Drive/after_data/20181016_1_135124_person.wav',
 '/content/drive/My Drive/after_data/20181009_1_103752_car.wav',
 '/content/drive/My Drive/after_data/20181016_2_134532_car.wav',
 '/content/drive/My Drive/after_data/20181009_1_104216_car.wav',
 '/content/drive/My Drive/after_data/20181016_3_142025_person.wav',
 '/content/drive/My Drive/after_data/20181009_1_104446_car.wav',
 '/content/drive/My Drive/after_data/20181016_3_142531_drone.wav',
 '/content/drive/My Drive/after_data/20181009_2_102248_person.wa

In [0]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield int(start), int(start + window_size)
        start += (window_size / 2)

def extract_features(data ,bands = 20, frames = 41):
    window_size = 512 * (frames - 1)
    mfccs = []
    labels = []
    log_specgrams = []
    sound_clip,s = librosa.load(data)
    label = int(data.split('/')[-1].split('_')[1])-1
    for (start,end) in windows(sound_clip,window_size):
        if(len(sound_clip[start:end]) == window_size):
            signal = sound_clip[start:end]
            melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
            logspec = librosa.amplitude_to_db(melspec)
            logspec = logspec.T.flatten()[:, np.newaxis].T
            #print(1, logspec.shape)
            log_specgrams.append(logspec)
            
            mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
            mfccs.append(mfcc)
            #print(2, mfcc.shape)
            features = np.hstack((mfccs, log_specgrams))
            labels.append(label)         
    features = np.asarray(features).reshape(len(mfccs), frames, bands*2)
    return np.array(features), np.array(labels,dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = 3
    one_hot_encode = np.zeros((n_labels, 3))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [0]:
n_wav = len(wav_data) 
stacked_features = []
stacked_labels = []

for i in range(n_wav) :
    tr_features, tr_labels = extract_features(wav_data[i])
    tr_labels = one_hot_encode(tr_labels)
    stacked_features.append(tr_features)
    stacked_labels.append(tr_labels)


In [61]:
stacked_features = np.array(stacked_features)
stacked_features.shape

(24,)

In [62]:
stacked_labels = np.array(stacked_labels)
stacked_labels.shape

(24,)

In [0]:
np.random.seed(1234)

X = stacked_features
y = stacked_labels

rnd_ind = np.random.permutation(len(X))

X = X[rnd_ind]
y = y[rnd_ind]

In [0]:
X_train = np.concatenate(X[:20], axis=0)
y_train = np.concatenate(y[:20], axis=0)

X_val = np.concatenate(X[20:], axis=0)
y_val = np.concatenate(y[20:], axis=0)


In [68]:
X_train.shape

(2041, 41, 40)

In [69]:
X_val.shape

(397, 41, 40)

keras model 

In [78]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation, Flatten, Reshape, LeakyReLU, ReLU
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential
import keras
import tensorflow as tf

Using TensorFlow backend.


In [0]:
tf.reset_default_graph()

learning_rate = 0.01
batch_size = 50
display_step = 200

# Network Parameters
n_input = 40 
n_steps = 41
n_hidden = 50
n_classes = 3 

In [85]:
#build model


model = Sequential()

model.add(LSTM(units=n_hidden, activation='tanh', input_shape=(n_steps, n_input),return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=False))
#model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 41, 50)            18200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 41, 50)            20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 41, 50)            20200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 41, 50)            20200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 41, 50)            20200     
_________________________________________________________________
lstm_5 (LSTM)                (None, 41, 50)            20200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 41, 50)            2

In [86]:
#callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc', patience=50),
#                  keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
#                                                       factor=0.1, patience=50)]

model.fit(X_train, y_train, 
          batch_size=batch_size,validation_data=(X_val, y_val), 
          epochs = 100, verbose=1)


Train on 2041 samples, validate on 397 samples
Epoch 1/100
2041/2041 [==============================] - 29s 14ms/sample - loss: 1.0129 - acc: 0.5105 - val_loss: 1.1097 - val_acc: 0.4736
Epoch 2/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 1.0118 - acc: 0.5355 - val_loss: 1.0246 - val_acc: 0.6045
Epoch 3/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 1.0585 - acc: 0.4370 - val_loss: 1.0727 - val_acc: 0.6045
Epoch 4/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 1.0392 - acc: 0.4375 - val_loss: 1.2195 - val_acc: 0.6045
Epoch 5/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 0.9969 - acc: 0.4434 - val_loss: 1.0892 - val_acc: 0.1965
Epoch 6/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 1.0286 - acc: 0.4385 - val_loss: 1.0015 - val_acc: 0.4433
Epoch 7/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 1.0075 - acc: 0.4517 - val_loss: 1.016

In [88]:
# drop out version 

if model in locals() :
  del model

model = Sequential()

model.add(LSTM(units=n_hidden, activation='tanh', input_shape=(n_steps, n_input),return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=True))
model.add(LSTM(units=n_hidden, activation='tanh', return_sequences=False))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 41, 50)            18200     
_________________________________________________________________
lstm_9 (LSTM)                (None, 41, 50)            20200     
_________________________________________________________________
lstm_10 (LSTM)               (None, 41, 50)            20200     
_________________________________________________________________
lstm_11 (LSTM)               (None, 41, 50)            20200     
_________________________________________________________________
lstm_12 (LSTM)               (None, 41, 50)            20200     
_________________________________________________________________
lstm_13 (LSTM)               (None, 41, 50)            20200     
_________________________________________________________________
lstm_14 (LSTM)               (None, 41, 50)           

In [89]:
#callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc', patience=50),
#                  keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
#                                                       factor=0.1, patience=50)]

model.fit(X_train, y_train, 
          batch_size=batch_size,validation_data=(X_val, y_val), 
          epochs = 100, verbose=1)

Train on 2041 samples, validate on 397 samples
Epoch 1/100
2041/2041 [==============================] - 30s 14ms/sample - loss: 1.0662 - acc: 0.4165 - val_loss: 1.0956 - val_acc: 0.6045
Epoch 2/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 1.0250 - acc: 0.4517 - val_loss: 1.0488 - val_acc: 0.2343
Epoch 3/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 0.9984 - acc: 0.4669 - val_loss: 0.8948 - val_acc: 0.6045
Epoch 4/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 0.9889 - acc: 0.4704 - val_loss: 1.0361 - val_acc: 0.6045
Epoch 5/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 0.9877 - acc: 0.5341 - val_loss: 0.9728 - val_acc: 0.3451
Epoch 6/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 0.9464 - acc: 0.5742 - val_loss: 0.9872 - val_acc: 0.3375
Epoch 7/100
2041/2041 [==============================] - 24s 12ms/sample - loss: 0.9745 - acc: 0.5247 - val_loss: 1.019